In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from google.colab import files
from difflib import get_close_matches

In [15]:
model = MobileNetV2(weights='imagenet')

In [27]:
uploaded = files.upload()  # Upload calories.csv here

df = pd.read_csv('calories.csv')
calorie_dict = dict(zip(df['Food'].str.lower(), df['Calories']))

Saving calories.csv to calories (2).csv


In [ ]:
uploaded = files.upload()

for file_name in uploaded.keys():
    img_path = file_name
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

preds = model.predict(img_array)
decoded = decode_predictions(preds, top=3)[0]

print("🍽️ Top Predictions:")
for i, (code, label, prob) in enumerate(decoded):
    print(f"{i+1}: {label} ({prob*100:.2f}%)")

# Step 2: Get the top predicted label
top_prediction = decoded[0][1].lower()

# Step 3: Calorie lookup logic
if top_prediction in calorie_dict:
    calories = calorie_dict[top_prediction]
    print(f"\n🔥 Estimated Calories for {top_prediction.title()}: {calories} kcal")
else:
    # Try fuzzy match
    match = get_close_matches(top_prediction, calorie_dict.keys(), n=1, cutoff=0.6)
    if match:
        calories = calorie_dict[match[0]]
        print(f"\n🔍 Using approximate match: {match[0].title()}")
        print(f"🔥 Estimated Calories for {match[0].title()}: {calories} kcal")
    else:
        print(f"\n⚠️ Calorie information not found for: {top_prediction.title()}")